# Double Pendulum system

Here, the system is composed of two pendulums of length $l_1,l_2$, and mass $m_1,m_2$ concentrated at one end, respectively. The pivot of the first pendulum is attached to a fix point, that of the second pendulum is attached to the dangling bob of the first pendulum. The system is submitted to a uniform, constant gravitational acceleration field $g$ and evolves without friction.

The generalised coordinates of the system are the angles $\theta_1,\theta_2$ of each pendulum with the vertical downward direction. The coordinates are then given by

\begin{equation*}
\begin{array}{rcl}
x_1 & = & l_1\sin\theta_1\\
y_1 & = & -l_1\cos\theta_1
\end{array}
\hspace{2cm}
\begin{array}{rcl}
x_2 & = & l_1\sin\theta_1+l_2\sin\theta_2\\
y_2 & = & -l_1\cos\theta_1-l_2\cos\theta_2
\end{array}
\end{equation*}

The only active forces are the weights of the form $-\nabla V(m_i,x_i,y_i)$ with $V(m,x,y)=mgy$. Hence the Lagrangian

\begin{eqnarray*}
\mathcal{L} & = &
\underbrace{
\frac{1}{2}m_1l_1^2\dot{\theta}_1^2+\frac{1}{2}m_2(l_1^2\dot{\theta}_1^2+l_2^2\dot{\theta}_2^2+
2l_1l_2\dot{\theta}_1\dot{\theta}_2\cos(\theta_1-\theta_2))
}_{\textrm{kinetic energy}}
+
\underbrace{
m_1gl_1\cos\theta_1+m_2g(l_1\cos\theta_1+l_2\cos\theta_2)
}_{-\textrm{potential energy}}\\
& = &
\frac{1}{2}ml_1^2\dot{\theta}_1^2+\frac{1}{2}m_2l_2^2\dot{\theta}_2^2+
m_2l_1l_2\dot{\theta}_1\dot{\theta}_2\cos\Delta+
mgl_1\cos\theta_1+m_2gl_2\cos\theta_2
\end{eqnarray*}

where $m\triangleq m_1+m_2$ and $\Delta\triangleq\theta_1-\theta_2$. After simplification, the equations of the dynamics are given by:

\begin{equation*}
\begin{array}{rrcl}
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\theta}_1} = \frac{\partial\mathcal{L}}{\partial\theta_1}\right]
\hspace{.2cm} &
ml_1\ddot{\theta}_1+m_2l_2\ddot{\theta}_2\cos\Delta
& = &
-m_2l_2\dot{\theta}_2^2\sin\Delta-mg\sin\theta_1
\\
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\theta}_2} = \frac{\partial\mathcal{L}}{\partial\theta_2}\right]
\hspace{.2cm} &
m_2l_1\ddot{\theta}_1\cos\Delta+m_2l_2\ddot{\theta}_2
& = &
m_2l_1\dot{\theta}_1^2\sin\Delta-m_2g\sin\theta_2
\end{array}
\end{equation*}

which rewrites as

\begin{equation*}
\left\{
\begin{array}{rcl}
ab\ddot{\theta}_1 + \ddot{\theta}_2\cos\Delta & = & u
\\
b\ddot{\theta}_1\cos\Delta + \ddot{\theta}_2 & = & v
\end{array}
\right.
\hspace{1cm}\textrm{where}\hspace{1cm}
\left|
\begin{array}{l}
a \triangleq \frac{m}{m_2}=1+\frac{m_1}{m_2}
\hspace{1cm}
b \triangleq \frac{l_1}{l_2}
\hspace{1cm}
c \triangleq \frac{g}{l_2}
\\
u \triangleq -\dot{\theta}_2^2\sin\Delta-ac\sin\theta_1
\hspace{1cm}
v \triangleq b\dot{\theta}_1^2\sin\Delta-c\sin\theta_2
\end{array}
\right.
\end{equation*}

The solution in $\ddot{\theta}_1,\ddot{\theta}_2$ is therefore

\begin{equation*}
\left\{
\begin{array}{rcl}
\ddot{\theta}_1 & = &
\frac{u-v\cos\Delta}{b(a-\cos^2\Delta)}
\\
\ddot{\theta}_2 & = &
\frac{av-u\cos\Delta}{a-\cos^2\Delta}
\end{array}
\right.
\end{equation*}

Documentation: [here](../doc/_build/html/odesimu.html)

In [ ]:
%autoreload 2
%pylab

import logging
logger = logging.getLogger()

from functools import partial
from ipyshow.odesimu.system import System
from ipyshow.odesimu.util import logger_hook

## DoublePendulum class

In [ ]:
#----------------------------------------------------------------------------------------------------
class DoublePendulum (System):
#----------------------------------------------------------------------------------------------------

    def __init__(self,L1,L2,M1,M2,G):
        """
:param G: gravitational acceleration, in m/s^2
:param L1: length of pendulum 1 in m
:param L2: length of pendulum 2 in m
:param M1: mass of pendulum 1 in kg
:param M2: mass of pendulum 2 in kg
        """
        self.L1, self.L2, self.M1, self.M2, self.G = L1, L2, M1, M2, G
        def main(t,state,a=1+M1/M2,b=L1/L2,c=G/L2):
            theta1,dtheta1,theta2,dtheta2 = state
            delta = theta1-theta2
            cosdelta, sindelta = cos(delta), sin(delta)
            u = -square(dtheta2)*sindelta-a*c*sin(theta1)
            v = b*square(dtheta1)*sindelta-c*sin(theta2)
            D = a-square(cosdelta)
            ddtheta1 = (u-v*cosdelta)/(D*b)
            ddtheta2 = (a*v-u*cosdelta)/D
            return array((dtheta1, ddtheta1, dtheta2, ddtheta2))
        self.main = main
        def fordisplay(state):
            theta1,w1,theta2,w2 = state
            x1 = L1*sin(theta1)
            y1 = -L1*cos(theta1)
            x2 = x1 + L2*sin(theta2)
            y2 = y1 - L2*cos(theta2)
            live = (x1,y1),(x2,y2)
            return live, live[1]
        self.fordisplay = fordisplay

    def display(self,ax,refsize=100.,**ka):
        """
:param refsize: average size in points^2 of the pendulum (and size of rotation axe)

The actual size of the two pendulums are computed around this average to reflect their mass ratio.
        """
        L = 1.05*(self.L1+self.L2)
        ax.set_xlim(-L,L)
        ax.set_ylim(-L,L)
        ax.scatter((0.,),(0.,),c='k',marker='o',s=refsize)
        m1,m2 = self.M1,self.M2
        r = clip(sqrt(m1/m2),1./refsize,refsize)
        sz = (refsize*r,refsize/r)
        diag_l, = ax.plot((),(),'k')
        diag_s = ax.scatter((),(),s=sz,marker='o',c=('b','r'))
        tail_l, = ax.plot((),(),'y')
        ax.set_title(r'trajectory:cahotic')
        def disp(t,live,tail):
            (x1,y1),(x2,y2) = live
            diag_l.set_data((0,x1,x2),(0,y1,y2))
            diag_s.set_offsets(((x1,y1),(x2,y2)))
            tail_l.set_data(tail[:,0],tail[:,1])
        return super(DoublePendulum,self).display(ax,disp,**ka)

    @staticmethod
    def makestate(theta1=None,w1=0.,theta2=None,w2=0.): return array((theta1,w1,theta2,w2))*pi/180.


Launcher
--------

System parameters:

* `M1,M2`: masses of the bobs (in kg) - pivots are assumed massless
* `L1,L2`: lengths of the pendulums (in m)
* `G`: gravitational acceleration (in m.sec$^{-2}$)

Launch parameters:

* `ini/theta1,theta2`: initial angles of the pendulums with downward vertical (in deg)
* `ini/w1,w2`: initial angular speeds (in deg.sec$^{-1}$)
* `srate`: simulation rate (in frames.sec$^{-1}$)
* `taild`: duration of shadow, ie. trace of previous states (in sec)
* `hooks`: list of display hooks (see doc)

In [ ]:
logger.setLevel(logging.WARN) # can be changed dynamically by logger_hook
syst = DoublePendulum(M1=3.,L1=4.,M2=1.,L2=2.,G=9.807,)
#syst = DoublePendulum(M1=3.,L1=1.,M2=1.,L2=2.,G=1.622,) # the same on the Moon ...

syst.launch(
    ini=syst.makestate(
        theta1=180.,
        theta2=179.,
        w1=45.,
        w2=-45.,
        ),
    maxtime=infty,
    srate=25.,
    taild=1,
    hooks=(partial(logger_hook,logger=logger),),
    animate=dict(
        repeat=False,
        ),
    )